In [ ]:
import socket
import struct
import time
import math

# 服务器配置
UDP_IP = "127.0.0.1"  # 如果UE5运行在本机，就用127.0.0.1
UDP_PORT = 8888       # 必须与UE5 UDPReceiver端口匹配
INTERVAL = 0.033      # 大约每秒30次更新

def main():
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

    print(f"正在启动UDP服务器，地址：{UDP_IP}:{UDP_PORT}")
    print("按Ctrl+C停止")

    counter = 0

    try:
        while True:
            counter += 1

            # 位置数据：在 XY 平面做圆周运动，Z 上下震荡
            radius = 100.0
            pos_x = radius * math.cos(counter * 0.05)
            pos_y = radius * math.sin(counter * 0.05)
            pos_z = 50.0 + 20.0 * math.sin(counter * 0.03)

            # 旋转数据(度数)
            rot_pitch = 0.0
            rot_yaw   = (counter * 2.0) % 360.0
            rot_roll  = 20.0 * math.sin(counter * 0.1)

            # 数据顺序： x, y, z, pitch, yaw, roll (小端浮点)
            data = struct.pack('<ffffff',
                               pos_x, pos_y, pos_z,
                               rot_pitch, rot_yaw, rot_roll)

            sock.sendto(data, (UDP_IP, UDP_PORT))

            # 每30次发送打印一次
            if counter % 30 == 0:
                print(f"已发送：位置({pos_x:.1f}, {pos_y:.1f}, {pos_z:.1f}), "
                      f"旋转({rot_pitch:.1f}, {rot_yaw:.1f}, {rot_roll:.1f})")

            time.sleep(INTERVAL)

    except KeyboardInterrupt:
        print("UDP服务器已停止")
    finally:
        sock.close()

if __name__ == "__main__":
    main()
